In [94]:
import json
import pandas as pd
import time
import gcsfs

In [95]:
from google.cloud import storage 
from google.cloud import bigquery as bq
from pathlib import Path
import os

In [96]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'peii-242723-111b55a1e645.json'

In [168]:
df = pd.read_csv('gs://impact-index-shared-resources/data/interim/2019-11-06/interim_entrepreneur_data.csv')
df

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id
0,Scott Brown,ColdSpark,226a627bc92415995985cbd94743276b,coldspark,6500000,1000.0,0,False,"Broomfield, CO","Denver-Aurora-Lakewood, CO",NaN,NaN,NaN,NaN
1,Robin Horwitz,Convo Communications,7d0675cefac592615e1cbb6c29fd403a,convo-communications,500000,100.0,0,False,"Austin, TX","Austin-Round Rock, TX",NaN,NaN,NaN,NaN
2,Jiren Parikh,SnapOne,ed5f8110a213395ececfe94660c0f602,snapone-inc,0,100.0,1,False,"Princeton, NJ","Trenton-Ewing, NJ",Male,NaN,NaN,NaN
3,Matthew Slipper,Symphony Communication Services,5f89826c5031a1932f27525b505b0a7f,symphony-3,461000000,500.0,1,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN
4,Ric Zhou,Kika Tech,437121710de6c1d5e8f8ebe555749fa8,kika-tech,63000000,500.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",NaN,NaN,NaN,NaN
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",Male,NaN,NaN,NaN
6,Sunil Agrawal,Armor5,3fca9b4b44b3ed61719f56c19af7387d,armor5,2000000,50.0,7,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN
7,Suzy Batiz,Poo~Pourri,3ac4350d0bd841a98beb24341e43a63f,poo-pourri,0,50.0,1,False,"Addison, TX","Dallas-Fort Worth-Arlington, TX",NaN,NaN,NaN,NaN
8,Brian Hastings,Gearbox Express,d4ed79c1d290badf8165832a8eed0d41,gearbox-express,2465000,50.0,0,False,"Mukwonago, WI","Milwaukee-Waukesha-West Allis, WI",Male,NaN,NaN,NaN
9,Brian Petersen,Podimetrics,8cded10bbd3be6f33dcf72367e5fb604,podimetrics,15900000,50.0,5,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",NaN,NaN,NaN,NaN


In [98]:
PROJ_ROOT = Path().resolve().parent
KEYS_DIR = PROJ_ROOT / 'keys' 
keys = KEYS_DIR / 'Keys for Big Query Storage Admin - PEII.json'

In [99]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(keys)

In [100]:
#this query gets only names that are female
f_names_query="""
#standardSQL
SELECT name, SUM(number) as records 
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE gender = 'F'
GROUP BY name 
ORDER BY records DESC
"""

job_config = bq.QueryJobConfig()
now=time.time()
query_job=client.query(f_names_query,location='US')
res=query_job.result()
print('query took:',round(time.time()-now,2),'s')

query took: 2.12 s


In [101]:
now=time.time()
f_names_df=res.to_dataframe()
print('load to pandas from BQ took:',round(time.time()-now,2),'s')

load to pandas from BQ took: 1.79 s


In [102]:
f_names = f_names_df['name'].to_list()
for i in range(len(f_names)):
    f_names[i] =  f_names[i].lower()

print(len(m_names))
f_names

13785


['mary',
 'patricia',
 'elizabeth',
 'jennifer',
 'linda',
 'barbara',
 'margaret',
 'susan',
 'dorothy',
 'jessica',
 'sarah',
 'nancy',
 'betty',
 'karen',
 'lisa',
 'helen',
 'sandra',
 'ashley',
 'kimberly',
 'emily',
 'donna',
 'carol',
 'michelle',
 'amanda',
 'melissa',
 'laura',
 'anna',
 'stephanie',
 'deborah',
 'rebecca',
 'ruth',
 'sharon',
 'cynthia',
 'kathleen',
 'amy',
 'shirley',
 'angela',
 'virginia',
 'catherine',
 'katherine',
 'brenda',
 'emma',
 'pamela',
 'nicole',
 'christine',
 'samantha',
 'rachel',
 'janet',
 'carolyn',
 'debra',
 'evelyn',
 'maria',
 'frances',
 'heather',
 'diane',
 'julie',
 'joyce',
 'martha',
 'alice',
 'victoria',
 'joan',
 'christina',
 'kelly',
 'lauren',
 'marie',
 'ann',
 'doris',
 'judith',
 'olivia',
 'jean',
 'cheryl',
 'megan',
 'kathryn',
 'andrea',
 'grace',
 'rose',
 'hannah',
 'jacqueline',
 'julia',
 'sara',
 'gloria',
 'teresa',
 'janice',
 'mildred',
 'theresa',
 'madison',
 'judy',
 'lillian',
 'beverly',
 'denise',
 'm

In [103]:
names_query="""
#standardSQL
SELECT name, SUM(number) as records 
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE gender = 'M'
GROUP BY name 
ORDER BY records DESC
"""

job_config = bq.QueryJobConfig()
now=time.time()
query_job=client.query(names_query,location='US')
res=query_job.result()
print('query took:',round(time.time()-now,2),'s')

query took: 2.45 s


In [104]:
now=time.time()
m_names_df=res.to_dataframe()
print('load to pandas from BQ took:',round(time.time()-now,2),'s')

load to pandas from BQ took: 1.65 s


In [105]:
m_names = m_names_df['name'].to_list()
for i in range(len(m_names)):
    m_names[i] =  m_names[i].lower()
    
print(len(m_names))
m_names

13785


['james',
 'john',
 'robert',
 'michael',
 'william',
 'david',
 'richard',
 'joseph',
 'charles',
 'thomas',
 'christopher',
 'daniel',
 'matthew',
 'anthony',
 'donald',
 'paul',
 'mark',
 'george',
 'steven',
 'andrew',
 'kenneth',
 'edward',
 'joshua',
 'kevin',
 'brian',
 'ronald',
 'timothy',
 'jason',
 'jeffrey',
 'ryan',
 'jacob',
 'gary',
 'nicholas',
 'eric',
 'stephen',
 'jonathan',
 'frank',
 'larry',
 'justin',
 'scott',
 'brandon',
 'raymond',
 'samuel',
 'benjamin',
 'gregory',
 'jack',
 'patrick',
 'alexander',
 'henry',
 'dennis',
 'jerry',
 'tyler',
 'aaron',
 'walter',
 'peter',
 'jose',
 'douglas',
 'adam',
 'nathan',
 'zachary',
 'harold',
 'arthur',
 'carl',
 'kyle',
 'albert',
 'gerald',
 'lawrence',
 'roger',
 'jeremy',
 'keith',
 'joe',
 'ethan',
 'terry',
 'christian',
 'sean',
 'willie',
 'austin',
 'jesse',
 'ralph',
 'noah',
 'billy',
 'bruce',
 'bryan',
 'roy',
 'jordan',
 'dylan',
 'louis',
 'eugene',
 'harry',
 'wayne',
 'alan',
 'russell',
 'juan',
 'ga

In [106]:
andro_names = (set(m_names).intersection(f_names))

print(len(andro_names))
andro_names

3042


{'rilee',
 'indigo',
 'mayan',
 'blakely',
 'marvel',
 'cyan',
 'patrick',
 'shalom',
 'kalei',
 'demetri',
 'walker',
 'clemence',
 'dayan',
 'maki',
 'wanda',
 'kinnick',
 'lawson',
 'chi',
 'ares',
 'jaylin',
 'alejandra',
 'alden',
 'mikel',
 'kameryn',
 'dru',
 'maggie',
 'scout',
 'rosio',
 'laakea',
 'meryl',
 'austyn',
 'sehaj',
 'notnamed',
 'loy',
 'izzy',
 'lizeth',
 'sheila',
 'bentley',
 'blayke',
 'adrain',
 'kainoa',
 'wesley',
 'christia',
 'camren',
 'martha',
 'sally',
 'aria',
 'leshawn',
 'van',
 'giselle',
 'lyn',
 'abbie',
 'jermiah',
 'jaydenn',
 'jess',
 'ariel',
 'sivan',
 'michell',
 'burnell',
 'elma',
 'kathy',
 'ricky',
 'jewel',
 'akari',
 'verlon',
 'pearly',
 'deshone',
 'carlton',
 'kem',
 'patty',
 'nathan',
 'jonah',
 'jacque',
 'rio',
 'juno',
 'gianni',
 'hillery',
 'danielle',
 'gussie',
 'maeson',
 'legacy',
 'amandeep',
 'francesca',
 'charley',
 'romaine',
 'braylyn',
 'newborn',
 'gurjot',
 'elijah',
 'oluwanifemi',
 'kelli',
 'tonnie',
 'love'

In [113]:
andro_names = list(andro_names)
andro_names = andro_names[1:]
andro_names[-1] = 'rilee'

In [ ]:
m_names_df = m_names_df.set_index('name')

In [62]:
m_names_df

,records
name,
James,4997327
John,4869607
Robert,4734038
Michael,4349307
William,3890923
David,3597725
Richard,2539873
Joseph,2522812
Charles,2273068


In [70]:
m_names_df.loc['Olivia']['records']

107

In [77]:
f_names_df = f_names_df.set_index('name')

In [78]:
f_names_df.loc['Olivia']['records']

446458

In [81]:
#temporarily remove the androgynous names from both lists
f_names = list(set(f_names) - set(m_names))
len(f_names)

m_names = list(set(m_names) - set(f_names))
len(m_names)

13785

In [114]:
andro_names

['mayan',
 'blakely',
 'marvel',
 'cyan',
 'patrick',
 'shalom',
 'kalei',
 'demetri',
 'walker',
 'clemence',
 'dayan',
 'maki',
 'wanda',
 'kinnick',
 'lawson',
 'chi',
 'ares',
 'jaylin',
 'alejandra',
 'alden',
 'mikel',
 'kameryn',
 'dru',
 'maggie',
 'scout',
 'rosio',
 'laakea',
 'meryl',
 'austyn',
 'sehaj',
 'notnamed',
 'loy',
 'izzy',
 'lizeth',
 'sheila',
 'bentley',
 'blayke',
 'adrain',
 'kainoa',
 'wesley',
 'christia',
 'camren',
 'martha',
 'sally',
 'aria',
 'leshawn',
 'van',
 'giselle',
 'lyn',
 'abbie',
 'jermiah',
 'jaydenn',
 'jess',
 'ariel',
 'sivan',
 'michell',
 'burnell',
 'elma',
 'kathy',
 'ricky',
 'jewel',
 'akari',
 'verlon',
 'pearly',
 'deshone',
 'carlton',
 'kem',
 'patty',
 'nathan',
 'jonah',
 'jacque',
 'rio',
 'juno',
 'gianni',
 'hillery',
 'danielle',
 'gussie',
 'maeson',
 'legacy',
 'amandeep',
 'francesca',
 'charley',
 'romaine',
 'braylyn',
 'newborn',
 'gurjot',
 'elijah',
 'oluwanifemi',
 'kelli',
 'tonnie',
 'love',
 'kiyomi',
 'munach

In [115]:
for i in range(len(andro_names)):
    andro_names[i] = andro_names[i].capitalize()
    print(andro_names[i])

Mayan
Blakely
Marvel
Cyan
Patrick
Shalom
Kalei
Demetri
Walker
Clemence
Dayan
Maki
Wanda
Kinnick
Lawson
Chi
Ares
Jaylin
Alejandra
Alden
Mikel
Kameryn
Dru
Maggie
Scout
Rosio
Laakea
Meryl
Austyn
Sehaj
Notnamed
Loy
Izzy
Lizeth
Sheila
Bentley
Blayke
Adrain
Kainoa
Wesley
Christia
Camren
Martha
Sally
Aria
Leshawn
Van
Giselle
Lyn
Abbie
Jermiah
Jaydenn
Jess
Ariel
Sivan
Michell
Burnell
Elma
Kathy
Ricky
Jewel
Akari
Verlon
Pearly
Deshone
Carlton
Kem
Patty
Nathan
Jonah
Jacque
Rio
Juno
Gianni
Hillery
Danielle
Gussie
Maeson
Legacy
Amandeep
Francesca
Charley
Romaine
Braylyn
Newborn
Gurjot
Elijah
Oluwanifemi
Kelli
Tonnie
Love
Kiyomi
Munachiso
Codie
Dasani
Puneet
Faith
Sequoia
Ellie
Carey
Haile
Kareem
Ilya
Carmel
Priscilla
Toi
Gerardo
Allyn
Onyx
Savanna
Valeria
Dorris
Oscar
Deven
Dustin
Laken
Tanay
Hilda
Tatiana
Ronald
Emily
Claire
Kairi
Anh
Victoria
Loraine
Kacy
Marvin
Clair
Catalina
Bayler
Shelley
Berkeley
Jamelle
Amarie
Samuel
Billye
Marilyn
Valencia
Drew
Darcy
Lindsay
Ethel
Benito
Josey
Jacqueline
L

Oddie
Pearl
Marco
Britt
Roma
Brendan
Macon
Daisy
Reginald
Caelan
Crystal
Matisse
Araceli
Delia
Eleanor
Elgie
Kaydence
Sade
Georgia
Calvin
Kobie
Skylor
Kamron
Brailyn
Ariana
Farron
Arnell
Haven
Jamari
Tobey
Rogue
Ruth
Brady
Robin
Murl
Eddie
Amare
Landen
Salvador
Darius
Roger
Uriah
Sai
Jillian
Gael
Carina
Beatriz
Alvis
Armani
Minh
Dorothy
Graysen
Ellington
Xiao
Kao
Levern
Arvis
Alan
Roni
Unnamed
Bryn
Kaylon
Selena
Enis
Parker
Soren
Erik
Jeffie
Cecilia
Eman
Johnathan
Kareen
Ynes
Naveen
Mckinley
Parrish
Angeles
Princess
Sevyn
Gary
Carrie
Bee
Lily
Gradie
Toy
Blaise
Harrison
Ayven
Nakia
Haley
Payden
Heather
Cali
Kylie
Luis
Lorin
Emory
Zen
Kevyn
Kayron
Lizette
Cleofas
Dean
Giovanni
Keith
Marice
Brecken
Murphy
Darion
Nichelle
Glen
Clayton
Eric
Karson
Vester
Lonnie
Annie
Kimani
Trenidad
Lavon
Kylen
James
Chee
Royal
Aiman
Millie
Yuval
Jordyn
Nasim
Lane
Isabel
Jaydin
Vaughn
Zyon
Jaydyn
Avon
Rachel
Kaden
Ahmari
Jamaria
Morgen
Luan
Jaquay
Justine
Travis
Gwyn
Tanveer
Russell
Sylvia
Tamika
Alex
Chels

In [123]:
m_names_df = m_names_df.set_index('name')
m_names_df

,records
name,
James,4997327
John,4869607
Robert,4734038
Michael,4349307
William,3890923
David,3597725
Richard,2539873
Joseph,2522812
Charles,2273068


In [121]:
f_names_df = f_names_df.set_index('name')

In [143]:
count_m = 0
count_f = 0
count_na = 0

for name in andro_names:
    male_count = m_names_df.loc[name]['records']
    female_count = f_names_df.loc[name]['records']
    total = male_count + female_count
    percentage_male = round(male_count / total, 2)
    percentage_female = round(female_count / total, 2)
    if percentage_male >= 0.75:
        gender_assign = 'M'
        count_m += 1
    elif percentage_female >= 0.75:
        gender_assign = 'F'
        count_f += 1
    else:
        gender_assign = 'N/A'
        count_na += 1
        
    x = 10 - len(str(male_count + female_count))
    y = 10 - len(name)
    print(name, y*' ', male_count, female_count, x*' ', ' | ', percentage_male, percentage_female, ' | ', gender_assign)
    
print('count M', count_m)
print('count_F', count_f)
print('count_na', count_na)
    
    
#     if male_count > female_count:
#         m_names.append(name)
#     if female_count > male_count:
#         f_names.append(name)

Mayan       5 14           |  0.26 0.74  |  N/A
Blakely     64 7442         |  0.01 0.99  |  F
Marvel      34 2326         |  0.01 0.99  |  F
Cyan        10 271          |  0.04 0.96  |  F
Patrick     666959 576       |  1.0 0.0  |  M
Shalom      1011 107         |  0.9 0.1  |  M
Kalei       40 536          |  0.07 0.93  |  F
Demetri     1582 16         |  0.99 0.01  |  M
Walker      20112 12        |  1.0 0.0  |  M
Clemence    15 16           |  0.48 0.52  |  N/A
Dayan       20 26           |  0.43 0.57  |  N/A
Maki        15 5           |  0.75 0.25  |  M
Wanda       58 276149       |  0.0 1.0  |  F
Kinnick     327 5          |  0.98 0.02  |  M
Lawson      8456 34         |  1.0 0.0  |  M
Chi         59 93          |  0.39 0.61  |  N/A
Ares        2237 5         |  1.0 0.0  |  M
Jaylin      8593 6387        |  0.57 0.43  |  N/A
Alejandra   280 48558        |  0.01 0.99  |  F
Alden       8840 43         |  1.0 0.0  |  M
Mikel       6307 17         |  1.0 0.0  |  M
Kameryn     54 703  

Kenny       46713 17        |  1.0 0.0  |  M
Ramsey      2302 343         |  0.87 0.13  |  M
Verna       11 57043        |  0.0 1.0  |  F
Kaitlin     6 55909        |  0.0 1.0  |  F
Kinta       11 6           |  0.65 0.35  |  N/A
Brennen     6624 6         |  1.0 0.0  |  M
Kathryn     35 433894       |  0.0 1.0  |  F
Marcy       6 17800        |  0.0 1.0  |  F
Otis        49972 64        |  1.0 0.0  |  M
Perla       6 17654        |  0.0 1.0  |  F
Taylin      23 900          |  0.02 0.98  |  F
Conley      969 28          |  0.97 0.03  |  M
Chai        55 11           |  0.83 0.17  |  M
Olivia      107 446458       |  0.0 1.0  |  F
Kevin       1168738 2557      |  1.0 0.0  |  M
Ivey        741 1392         |  0.35 0.65  |  N/A
Oakley      3594 3412         |  0.51 0.49  |  N/A
Shelly      297 81628        |  0.0 1.0  |  F
Phuc        47 5           |  0.9 0.1  |  M
Chanel      26 14567        |  0.0 1.0  |  F
Koi         37 13           |  0.74 0.26  |  N/A
Callahan    395 5          | 

Yancy       452 55          |  0.89 0.11  |  M
Sydney      3953 165968       |  0.02 0.98  |  F
Shonte      10 202          |  0.05 0.95  |  F
Jaelen      410 13          |  0.97 0.03  |  M
Misty       19 82192        |  0.0 1.0  |  F
Lilly       6 49743        |  0.0 1.0  |  F
Alyssa      70 305332       |  0.0 1.0  |  F
Kailen      61 53          |  0.54 0.46  |  N/A
Amari       17101 13479        |  0.56 0.44  |  N/A
Kyler       28983 579        |  0.98 0.02  |  M
Bennett     35896 388        |  0.99 0.01  |  M
Davy        532 5          |  0.99 0.01  |  M
Kiara       93 47587        |  0.0 1.0  |  F
Nuri        5 87           |  0.05 0.95  |  F
Jane        42 358170       |  0.0 1.0  |  F
Weston      42396 5        |  1.0 0.0  |  M
Patrice     136 30932        |  0.0 1.0  |  F
Brennan     29197 428        |  0.99 0.01  |  M
Amelia      18 178299       |  0.0 1.0  |  F
Claudie     1478 1882         |  0.44 0.56  |  N/A
Nora        12 125332       |  0.0 1.0  |  F
Teddie      60 37  

Delane      5 20           |  0.2 0.8  |  F
Cameryn     48 1269         |  0.04 0.96  |  F
Suzanne     5 215433       |  0.0 1.0  |  F
Abby        14 55227        |  0.0 1.0  |  F
Pearlie     66 14151        |  0.0 1.0  |  F
Ava         50 260436       |  0.0 1.0  |  F
Vada        15 5708         |  0.0 1.0  |  F
Briar       2358 1621         |  0.59 0.41  |  N/A
Tru         301 209          |  0.59 0.41  |  N/A
Riki        41 117          |  0.26 0.74  |  N/A
Ladell      27 25           |  0.52 0.48  |  N/A
Shia        839 176         |  0.83 0.17  |  M
Encarnacion  543 29          |  0.95 0.05  |  M
Marie       183 463763       |  0.0 1.0  |  F
Andree      186 438          |  0.3 0.7  |  N/A
Carter      153737 4112       |  0.97 0.03  |  M
Jourdan     501 983         |  0.34 0.66  |  N/A
Miller      3334 251         |  0.93 0.07  |  M
Mylan       129 5          |  0.96 0.04  |  M
Jessi       433 3528         |  0.11 0.89  |  F
Clare       1432 23182        |  0.06 0.94  |  F
Yue     

Schuyler    1187 208         |  0.85 0.15  |  M
Homer       49458 5        |  1.0 0.0  |  M
Acie        229 5          |  0.98 0.02  |  M
Eduardo     101136 193       |  1.0 0.0  |  M
Bruce       379065 35       |  1.0 0.0  |  M
Kimi        144 345          |  0.29 0.71  |  N/A
Shanon      392 2590         |  0.13 0.87  |  F
Evon        7 906          |  0.01 0.99  |  F
Quin        143 21          |  0.87 0.13  |  M
Hiawatha    38 5           |  0.88 0.12  |  M
Ali         24470 7289        |  0.77 0.23  |  M
Merced      269 60          |  0.82 0.18  |  M
Harlyn      7 487          |  0.01 0.99  |  F
Nikita      1142 7997         |  0.12 0.88  |  F
Shawnta     5 743          |  0.01 0.99  |  F
Leticia     59 38724        |  0.0 1.0  |  F
Atley       15 53           |  0.22 0.78  |  F
Mason       278779 1417       |  0.99 0.01  |  M
Shaune      5 5           |  0.5 0.5  |  N/A
Esther      33 215997       |  0.0 1.0  |  F
Hinckley    12 5           |  0.71 0.29  |  N/A
Tiffany     311 33

...

Shilo       48 300          |  0.14 0.86  |  F
Pressley    5 64           |  0.07 0.93  |  F
Mariela     5 8839         |  0.0 1.0  |  F
Alfred      222579 60       |  1.0 0.0  |  M
Thanh       378 183          |  0.67 0.33  |  N/A
Estell      61 2217         |  0.03 0.97  |  F
Daniela     37 58187        |  0.0 1.0  |  F
Ever        2066 582         |  0.78 0.22  |  M
Kriss       10 5           |  0.67 0.33  |  N/A
Rebecca     315 736536       |  0.0 1.0  |  F
Jeremy      434497 852       |  1.0 0.0  |  M
Christan    299 271          |  0.52 0.48  |  N/A
Dyllan      1624 15         |  0.99 0.01  |  M
Royce       22215 315        |  0.99 0.01  |  M
Wilfred     23225 5        |  1.0 0.0  |  M
Abigail     114 363136       |  0.0 1.0  |  F
Osie        32 168          |  0.16 0.84  |  F
Aydan       4162 10         |  1.0 0.0  |  M
Edlin       5 174          |  0.03 0.97  |  F
Karter      15874 1823        |  0.9 0.1  |  M
Roberto     105394 206       |  1.0 0.0  |  M
Kathleen    296 704489

Avion       879 19          |  0.98 0.02  |  M
Virgie      27 14374        |  0.0 1.0  |  F
Daryan      32 6           |  0.84 0.16  |  M
Zion        27650 5088        |  0.84 0.16  |  M
Glynn       2586 5         |  1.0 0.0  |  M
Nikki       47 36221        |  0.0 1.0  |  F
Gordie      10 5           |  0.67 0.33  |  N/A
Sascha      22 185          |  0.11 0.89  |  F
Kamani      47 177          |  0.21 0.79  |  F
Canaan      2595 5         |  1.0 0.0  |  M
Lonie       41 279          |  0.13 0.87  |  F
Tonya       5 117859       |  0.0 1.0  |  F
Terrell     31753 122        |  1.0 0.0  |  M
Serenity    7 56031        |  0.0 1.0  |  F
Beckett     14439 150        |  0.99 0.01  |  M
Toryn       10 5           |  0.67 0.33  |  N/A
Daryn       301 97          |  0.76 0.24  |  M
Robyn       29 59351        |  0.0 1.0  |  F
Lydia       28 149398       |  0.0 1.0  |  F
Sofia       5 131279       |  0.0 1.0  |  F
Ari         12229 1972        |  0.86 0.14  |  M
Toby        29287 4925        |

Nhia        91 7           |  0.93 0.07  |  M
Nikola      1236 15         |  0.99 0.01  |  M
Tierra      27 13001        |  0.0 1.0  |  F
Jentry      6 206          |  0.03 0.97  |  F
Atlas       4449 37         |  0.99 0.01  |  M
Chia        23 35           |  0.4 0.6  |  N/A
Davis       23637 42        |  1.0 0.0  |  M
Lacy        3510 16203        |  0.18 0.82  |  F
Selma       6 16986        |  0.0 1.0  |  F
Bertie      6 7441         |  0.0 1.0  |  F
Garnet      323 3857         |  0.08 0.92  |  F
Ashlee      12 41702        |  0.0 1.0  |  F
Bobby       310744 6434       |  0.98 0.02  |  M
Jason       1029719 3450      |  1.0 0.0  |  M
Kylin       393 186          |  0.68 0.32  |  N/A
Johnnie     93262 43015       |  0.68 0.32  |  N/A
Jensyn      5 37           |  0.12 0.88  |  F
Donzell     82 7           |  0.92 0.08  |  M
Hudsyn      5 39           |  0.11 0.89  |  F
Mannie      101 17          |  0.86 0.14  |  M
Jessica     2102 1043442      |  0.0 1.0  |  F
Min         38 12 

Elza        534 10          |  0.98 0.02  |  M
Hunter      231994 10429       |  0.96 0.04  |  M
Alexander   664801 2410       |  1.0 0.0  |  M
Dewey       26806 5        |  1.0 0.0  |  M
Diamante    73 5           |  0.94 0.06  |  M
Khali       37 289          |  0.11 0.89  |  F
Amrit       138 159          |  0.46 0.54  |  N/A
Gagandeep   57 15           |  0.79 0.21  |  M
Miguel      173683 400       |  1.0 0.0  |  M
Haylee      8 27094        |  0.0 1.0  |  F
Charles     2273068 7532      |  1.0 0.0  |  M
Lawrence    441700 102       |  1.0 0.0  |  M
Stephen     851239 721       |  1.0 0.0  |  M
Darrien     1376 19         |  0.99 0.01  |  M
Giani       5 10           |  0.33 0.67  |  N/A
Terran      67 5           |  0.93 0.07  |  M
Caroline    44 214033       |  0.0 1.0  |  F
Saylor      17 4469         |  0.0 1.0  |  F
Majesty     39 560          |  0.07 0.93  |  F
Ysabel      104 499          |  0.17 0.83  |  F
Aldean      12 5           |  0.71 0.29  |  N/A
Quinn       28742 2

Carl        478734 322       |  1.0 0.0  |  M
Arian       3391 174         |  0.95 0.05  |  M
Marty       28233 1534        |  0.95 0.05  |  M
Rayn        16 10           |  0.62 0.38  |  N/A
Keaton      18212 184        |  0.99 0.01  |  M
Kaylan      17 2576         |  0.01 0.99  |  F
Daniele     125 1577         |  0.07 0.93  |  F
Jia         112 600          |  0.16 0.84  |  F
Nabil       569 18          |  0.97 0.03  |  M
Kaelin      133 1145         |  0.1 0.9  |  F
Keyon       3774 6         |  1.0 0.0  |  M
Pedro       84488 124        |  1.0 0.0  |  M
Makell      6 51           |  0.11 0.89  |  F
Karina      110 56055        |  0.0 1.0  |  F
Adam        549029 622       |  1.0 0.0  |  M
Will        25394 65        |  1.0 0.0  |  M
Carsen      1951 78         |  0.96 0.04  |  M
Armando     69597 129        |  1.0 0.0  |  M
Harlow      287 5672         |  0.05 0.95  |  F
Juana       18 13540        |  0.0 1.0  |  F
Jayme       1053 10206        |  0.09 0.91  |  F
Erie        308 

In [186]:
andro_names2 = []

#identify which name from the androgynous list can be assigned to each gender
threshhold = 0.75
for name in andro_names:
    male_count = m_names_df.loc[name]['records']
    female_count = f_names_df.loc[name]['records']
    total = male_count + female_count
    percentage_male = round(male_count / total, 2)
    percentage_female = round(female_count / total, 2)
    
    #will only assign to a gender if the percentage is above our threshold level
    if percentage_male >= threshhold:
        m_names.append(name)
    elif percentage_female >= threshhold:
        f_names.append(name)
    else:
        andro_names2.append(name)

print('m_names', len(set(m_names)))
print('f_names', len(set(f_names)))
print('f_names', len(set(andro_names)))
   

m_names 15057
f_names 22620
f_names 3040


In [187]:
#using a set because I appended twice
m_names = len(set(m_names))

In [188]:
f_names = len(set(f_names))

In [169]:
#creating the first name column
for i in df.index:
    df.at[i, 'f_name'] = df.at[i, 'name'].split(' ')[0]

In [150]:
df

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name
0,Scott Brown,ColdSpark,226a627bc92415995985cbd94743276b,coldspark,6500000,1000.0,0,False,"Broomfield, CO","Denver-Aurora-Lakewood, CO",NaN,NaN,NaN,NaN,Scott
1,Robin Horwitz,Convo Communications,7d0675cefac592615e1cbb6c29fd403a,convo-communications,500000,100.0,0,False,"Austin, TX","Austin-Round Rock, TX",NaN,NaN,NaN,NaN,Robin
2,Jiren Parikh,SnapOne,ed5f8110a213395ececfe94660c0f602,snapone-inc,0,100.0,1,False,"Princeton, NJ","Trenton-Ewing, NJ",Male,NaN,NaN,NaN,Jiren
3,Matthew Slipper,Symphony Communication Services,5f89826c5031a1932f27525b505b0a7f,symphony-3,461000000,500.0,1,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Matthew
4,Ric Zhou,Kika Tech,437121710de6c1d5e8f8ebe555749fa8,kika-tech,63000000,500.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",NaN,NaN,NaN,NaN,Ric
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",Male,NaN,NaN,NaN,Kumaran
6,Sunil Agrawal,Armor5,3fca9b4b44b3ed61719f56c19af7387d,armor5,2000000,50.0,7,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Sunil
7,Suzy Batiz,Poo~Pourri,3ac4350d0bd841a98beb24341e43a63f,poo-pourri,0,50.0,1,False,"Addison, TX","Dallas-Fort Worth-Arlington, TX",NaN,NaN,NaN,NaN,Suzy
8,Brian Hastings,Gearbox Express,d4ed79c1d290badf8165832a8eed0d41,gearbox-express,2465000,50.0,0,False,"Mukwonago, WI","Milwaukee-Waukesha-West Allis, WI",Male,NaN,NaN,NaN,Brian
9,Brian Petersen,Podimetrics,8cded10bbd3be6f33dcf72367e5fb604,podimetrics,15900000,50.0,5,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",NaN,NaN,NaN,NaN,Brian


In [172]:
null = df[pd.isnull(df['gender'])]
null.shape

(0, 15)

In [192]:
f_names

22620

In [199]:
f_names

22620

In [194]:
'Brian' in list(f_names)

TypeError: 'int' object is not iterable

In [171]:
count = 0

for i in null.index:
    name = null.at[i, 'f_name'].lower()
    
    if name in f_names:
        df.at[i, 'gender'] = 'f'
        count += 1
        
    elif name in m_names:
        df.at[i, 'gender'] = 'm'
        
    elif name in andro_names2:
        df.at[i, 'gender'] = 'andro'
        
    else:
        df.at[i, 'gender'] = 'i donno'
        
print(count)

17162


In [173]:
df

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name
0,Scott Brown,ColdSpark,226a627bc92415995985cbd94743276b,coldspark,6500000,1000.0,0,False,"Broomfield, CO","Denver-Aurora-Lakewood, CO",f,NaN,NaN,NaN,Scott
1,Robin Horwitz,Convo Communications,7d0675cefac592615e1cbb6c29fd403a,convo-communications,500000,100.0,0,False,"Austin, TX","Austin-Round Rock, TX",f,NaN,NaN,NaN,Robin
2,Jiren Parikh,SnapOne,ed5f8110a213395ececfe94660c0f602,snapone-inc,0,100.0,1,False,"Princeton, NJ","Trenton-Ewing, NJ",Male,NaN,NaN,NaN,Jiren
3,Matthew Slipper,Symphony Communication Services,5f89826c5031a1932f27525b505b0a7f,symphony-3,461000000,500.0,1,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Matthew
4,Ric Zhou,Kika Tech,437121710de6c1d5e8f8ebe555749fa8,kika-tech,63000000,500.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",m,NaN,NaN,NaN,Ric
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",Male,NaN,NaN,NaN,Kumaran
6,Sunil Agrawal,Armor5,3fca9b4b44b3ed61719f56c19af7387d,armor5,2000000,50.0,7,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Sunil
7,Suzy Batiz,Poo~Pourri,3ac4350d0bd841a98beb24341e43a63f,poo-pourri,0,50.0,1,False,"Addison, TX","Dallas-Fort Worth-Arlington, TX",f,NaN,NaN,NaN,Suzy
8,Brian Hastings,Gearbox Express,d4ed79c1d290badf8165832a8eed0d41,gearbox-express,2465000,50.0,0,False,"Mukwonago, WI","Milwaukee-Waukesha-West Allis, WI",Male,NaN,NaN,NaN,Brian
9,Brian Petersen,Podimetrics,8cded10bbd3be6f33dcf72367e5fb604,podimetrics,15900000,50.0,5,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",f,NaN,NaN,NaN,Brian


In [184]:
# Create column 'female'
df['female'] = 0

# Collect indices of female and androgynous names, and mark them as female
female_names = []
andy_names = []
unk_names = []

count_female = 0
count_andy = 0
count_unk = 0

for i in df.index:
    name = df.at[i, 'f_name']
    if d.get_gender(name) == 'female':
        female_names.append([i, name])
        df.at[i, 'female'] = 1
        count_female += 1
    elif d.get_gender(name) == 'mostly_female':
        female_names.append([i, name])
        df.at[i, 'female'] = 1
        count_female += 1
    elif d.get_gender(name) == 'andy':
        andy_names.append([i, name])
        count_andy += 1
    elif d.get_gender(name) == 'unknown':
        #if name in additional_female_names:
#             female_names.append([i, name])
#             df.at[i, 'female'] = 1
#             count_female += 1
#         #elif name in additional_male_names:
# #             pass
#         else:
#             unk_names.append([i, name])
            count_unk += 1

print(count_female, 'female names.')
print(count_andy, 'androgynous names.')
print(count_unk, 'unknown names.')

5439 female names.
693 androgynous names.
5040 unknown names.


In [185]:
df

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name,female
0,Scott Brown,ColdSpark,226a627bc92415995985cbd94743276b,coldspark,6500000,1000.0,0,False,"Broomfield, CO","Denver-Aurora-Lakewood, CO",f,NaN,NaN,NaN,Scott,0
1,Robin Horwitz,Convo Communications,7d0675cefac592615e1cbb6c29fd403a,convo-communications,500000,100.0,0,False,"Austin, TX","Austin-Round Rock, TX",f,NaN,NaN,NaN,Robin,0
2,Jiren Parikh,SnapOne,ed5f8110a213395ececfe94660c0f602,snapone-inc,0,100.0,1,False,"Princeton, NJ","Trenton-Ewing, NJ",Male,NaN,NaN,NaN,Jiren,0
3,Matthew Slipper,Symphony Communication Services,5f89826c5031a1932f27525b505b0a7f,symphony-3,461000000,500.0,1,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Matthew,0
4,Ric Zhou,Kika Tech,437121710de6c1d5e8f8ebe555749fa8,kika-tech,63000000,500.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",m,NaN,NaN,NaN,Ric,0
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",Male,NaN,NaN,NaN,Kumaran,0
6,Sunil Agrawal,Armor5,3fca9b4b44b3ed61719f56c19af7387d,armor5,2000000,50.0,7,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Sunil,0
7,Suzy Batiz,Poo~Pourri,3ac4350d0bd841a98beb24341e43a63f,poo-pourri,0,50.0,1,False,"Addison, TX","Dallas-Fort Worth-Arlington, TX",f,NaN,NaN,NaN,Suzy,1
8,Brian Hastings,Gearbox Express,d4ed79c1d290badf8165832a8eed0d41,gearbox-express,2465000,50.0,0,False,"Mukwonago, WI","Milwaukee-Waukesha-West Allis, WI",Male,NaN,NaN,NaN,Brian,0
9,Brian Petersen,Podimetrics,8cded10bbd3be6f33dcf72367e5fb604,podimetrics,15900000,50.0,5,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",f,NaN,NaN,NaN,Brian,0


In [164]:
print(df[ df['gender'] == 'i donno'].shape)
df[ df['gender'] == 'i donno']

(5590, 15)


,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",i donno,NaN,NaN,NaN,Kumaran
18,Chin Beckmann,DSP Concepts,deb1ab07d9a538dcdeeaf8f32f018d34,dsp-concepts,10000000,50.0,0,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",i donno,NaN,NaN,NaN,Chin
35,Yet-Ming Chiang,Form Energy,3b8abc89b8594793b6b4c4273880e0e8,form-energy,49000000,50.0,0,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",i donno,NaN,NaN,NaN,Yet-Ming
47,Zor Gorelov,Kasisto,f91becc6559dc2c44e815f6f94526e17,kasisto-inc,43450000,100.0,0,False,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",i donno,NaN,NaN,NaN,Zor
59,Charles-Albert Gorra,Trendlee,3d69dea93303ddecc5fb4213e92d9219,trendlee,29775093,50.0,0,False,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",i donno,NaN,NaN,NaN,Charles-Albert
68,Tingfan Wu,Umbo Computer Vision,4070b2d86ee801dc443c0c36883f1e10,umbo-computer-vision,17600000,50.0,0,False,"San Francisco, CA","San Francisco-Oakland-Hayward, CA",i donno,NaN,NaN,NaN,Tingfan
76,Thabet Amer,Souktel,92df209ba055cd01e350bfa87839e6f9,souktel,2965708,50.0,0,False,"Newark, DE","Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",i donno,NaN,NaN,NaN,Thabet
106,Wooshik Jung,"Stratio, Inc.",cdc2715a0fb79c82079f247ef732749e,stratio,4295000,50.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",i donno,NaN,NaN,NaN,Wooshik
144,Lodewijk Bogaards,StackState,4c8a7b41e4f0c6987f7cbc5ece4c0c53,stackstate,5000000,50.0,0,False,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",i donno,NaN,NaN,NaN,Lodewijk
157,Ofer Smadari,Luminate Security,6a999d47e65245f1b9bb0dbc52a23b92,luminate-security,14000000,50.0,0,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",i donno,NaN,NaN,NaN,Ofer


In [201]:
import numpy as np

In [203]:
percent_fem = np.round((count_female / (df.shape[0] - (count_andy + count_unk))), 2)
print(f'{int(percent_fem*100)}% of gender-identified entrepreneurs are female.')

12% of gender-identified entrepreneurs are female.


In [165]:
import gender_guesser.detector as gender

In [166]:
d = gender.Detector()

In [179]:
d.get_gender('Chantal')

'female'

In [59]:
df[df.f_name.isnull() == True]

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name


In [ ]:
for i in df.index:
    first = df.at[i, 'f_name']
    if 

In [ ]:
name_gender = {}

for i in social_security_df.index:
    social_security_name = social_security_df.at[i, 'name']
    social_security_gender = social_security_df.at[i, 'name']
    name_gender[social]
    for j in social_security_df.index:
        
        if name == social_security_name:
            print('true')

In [5]:
project = 'peii-242723'
dataset_id = 'entrepre'
table_id = 'Combined_tables_39k'
bucket_name = 'muse-data'

# Setup Storage vars
storage_client = storage.Client(project=project)
bucket = storage_client.get_bucket('muse-data')

# Setup BigQuery vars
bq_client = bigquery.Client()
dataset_ref = bq_client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)

#destination
destination_uri = "gs://{}/{}".format(bucket_name, "combined_tables_raw.json")

In [7]:
#set job configuration to extract job as JSON files
job_config = bigquery.ExtractJobConfig()
job_config.destination_format = 'NEWLINE_DELIMITED_JSON'
job_config.write_disposition = 'WRITE_TRUNCATE'

In [8]:
extract_job = bq_client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
    job_config=job_config
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)

Exported smart-science:Learners_Questions.Combined_Tables to gs://muse-data/combined_tables_raw.json


In [9]:
df = pd.read_json(destination_uri, lines=True)

In [10]:
df.shape

(32687, 11)

In [11]:
df

,answer,answer_type,last_modified,learner,learner_name,metatag,name,question,tag,text,who
0,NaN,binary,2017-06-04 15:56:53.69455 UTC,4535845151309824,Carolina,[answer],UFqE8BiENzhk07YcA0oD8Eaxrb50LSts,368,[yes],Do you have reliable childcare for {kid_name}?,parent
1,NaN,binary,2017-06-04 15:56:57.46436 UTC,4535845151309824,Carolina,[answer],kLHlljhhWAnDEAixy2iaR9QkNhPYgdiw,371,[yes],Do you work outside of the home?,parent
2,NaN,binary,2017-08-31 11:44:55.22537 UTC,4535845151309824,Carolina,[answer],AI9iuqtEMakSt0EqPasH4B9MxQuDgice,38,[definitely],Did you graduate from high school?,parent
3,1.0,binary,2017-01-15 01:09:13.02664 UTC,4536217521618944,Katie,[answer],zUXTth86bHSuv5tzk3HZP3pyt2cj7Zhp,22,[yes],Do you know where the nearest playground is fr...,parent
4,1.0,binary,2017-01-15 01:11:47.43045 UTC,4536217521618944,Katie,[answer],cjTk7pkKA37b4TAOuDl1gO5n0JUxOW4f,22,[yes],Do you know where the nearest playground is fr...,parent
5,1.0,scale,2017-01-15 01:09:10.63448 UTC,4536217521618944,Katie,[answer],fF1jcJpeFa0QNrMZDLDDYfvHT3iT5OeD,236,[yes],"Does {kid_name} complete familiar books, espec...",child
6,1.0,scale,2017-01-15 01:11:49.84119 UTC,4536217521618944,Katie,[answer],iFDQCB2EFajl0ZOJRWxVFb2Stg7trWHb,283,[yes],Does {kid_name} enjoy trying new things?,child
7,1.0,scale,2017-01-15 01:08:00.33935 UTC,4536217521618944,Katie,[answer],qiLwtrCvLnM6nDyuHMyV0SieyuYxFtlq,285,[yes],Does {kid_name} enjoy creating elaborate prete...,child
8,0.0,binary,2017-01-15 01:13:38.63336 UTC,4536217521618944,Katie,[answer],P5apuN9tKCPat5dhGPvWCz6WIEeofC0s,41,[no],Are you the first person in your family to gra...,parent
9,1.0,scale,2017-01-15 01:07:52.71889 UTC,4536217521618944,Katie,[answer],wlbqIQAZcupMEcczJxbCGwX2QiNKJogQ,434,[yes],Does {kid_name} usually share with others?,child


In [117]:
df = df[df.columns[df.isnull().all()]]
df

""
0
1
2
3
4
5
6
7
8
9


In [14]:
df['learner'].drop_duplicates()

0        4535845151309824
3        4536217521618944
23       4536398325481472
25       4551846551093248
30       4554084669456384
42       4563324725362688
44       4570824568733696
48       4576288907984896
54       4594617454428160
60       4597232309370880
62       4611123475120128
65       4613797645910016
67       4619529606922240
70       4623601126866944
75       4628968502198272
77       4631671437524992
79       4631909271339008
81       4640195836116992
147      4643024344711168
163      4643216242507776
166      4646135276765184
168      4649037902905344
183      4649666666823680
219      4651483148582912
222      4654768731455488
224      4657658128760832
226      4657999083732992
229      4659795990675456
239      4664326778519552
241      4666370654470144
               ...       
32550    6577171304808448
32554    6577703167721472
32558    6579174856720384
32560    6580381545398272
32562    6582805790195712
32564    6583070106845184
32565    6592382661296128
32575    659

2734 in 'Learners' SQL

In [17]:
df['last_modified'] = pd.to_datetime(df['last_modified'])
least_recent_date = df['last_modified'].min()
recent_date = df['last_modified'].max()

In [18]:
recent_date

Timestamp('2018-11-28 03:55:30.195524+0000', tz='UTC')

In [21]:
latest = df.loc[df.groupby('name').last_modified.idxmax()]

In [23]:
latest.shape

(16120, 11)

In [28]:
lastest = df.sort_values('last_modified').groupby('name').tail(1)

In [26]:
lastest.shape

(16120, 11)